# [TikAPI](https://tikapi.io/documentation/) user posts

#### Load Python tools and Jupyter config

In [1]:
import os
import json
import requests
import pandas as pd
import jupyter_black
import altair as alt
import altair_stiles as altstiles
from datetime import datetime
from tqdm.notebook import tqdm
from tikapi import TikAPI

In [2]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('stiles')

In [3]:
mykey = os.environ.get("tikapikey")
api = TikAPI(mykey)
today = pd.Timestamp("today").strftime("%Y%m%d")
user_unique_id = "famousmisspeaches"
user_name = "Miss Peaches"
color = "#fe2c55"

---

## Read

#### A user's TikTok posts into a dataframe

In [4]:
df = pd.read_csv(f"data/processed/{user_unique_id}_all_posts.csv").sort_values(
    "create_time_est"
)

In [5]:
df.head(1)

,user_id,user_name,user_unique_id,post_desc,post_id,duration,bookmark_count,comment_count,heart_count,share_count,play_count,create_time_est,created_date,created_time
97,7335986578207949870,Miss Peaches,famousmisspeaches,Meeting my papa for the 1st time,7335992056973643051,242,303,206,23800,1189,381700,2024-02-15 19:36:16-05:00,2024-02-15,19:36:16


In [6]:
numcols = [
    "duration",
    "bookmark_count",
    "comment_count",
    "heart_count",
    "share_count",
    "play_count",
]

---

## Process

#### Dates

In [7]:
df["month"] = pd.to_datetime(df["created_date"]).dt.month_name()
df["week"] = pd.to_datetime(df["created_date"]).dt.isocalendar().week
df["weekday"] = pd.to_datetime(df["created_date"]).dt.dayofweek

#### Normalize engagement measures

In [8]:
now = pd.to_datetime(datetime.now().strftime("%Y-%m-%d"))
df["days_since_posted"] = (now - pd.to_datetime(df["created_date"])).dt.days
df.loc[df["days_since_posted"] == 0, "days_since_posted"] = 1

#### Create new normalized columns with days

In [9]:
df["hearts_per_day"] = (df["heart_count"] / df["days_since_posted"]).round()

#### Create new normalized columns with plays

In [10]:
df["likes_per_play"] = df["heart_count"] / df["play_count"]
df["comments_per_play"] = df["comment_count"] / df["play_count"]
df["shares_per_play"] = df["share_count"] / df["play_count"]

#### Cumulative plays

In [11]:
df["play_count_cumsum"] = df["play_count"].cumsum()

#### Cumulative posts

In [12]:
df["single_post"] = 1
df["cumulative_post_count"] = df["single_post"].cumsum()
df = df.drop(columns=["single_post"])

---

## Aggregate

#### Posts per day

In [13]:
daily_counts = (
    df.groupby("created_date")["post_id"]
    .count()
    .reset_index()
    .rename(columns={"post_id": "post_count"})
)

In [14]:
daily_counts["date"] = pd.to_datetime(daily_counts["created_date"])

In [15]:
daily_counts["year"] = daily_counts["date"].dt.year
daily_counts["month"] = daily_counts["date"].dt.month
daily_counts["day"] = daily_counts["date"].dt.day
daily_counts["day_of_week"] = daily_counts["date"].dt.dayofweek
daily_counts["week_of_year"] = daily_counts["date"].dt.isocalendar().week

In [27]:
# Define the base of the chart
base = alt.Chart(daily_counts).encode(
    alt.X("week_of_year:O", title="Week of Year"),
    alt.Y("day_of_week:O", title="Day of Week"),
)

# Create the heatmap
heatmap = base.mark_rect().encode(color="post_count:Q", tooltip=["date", "post_count"])

# Add text to each cell
text = base.mark_text(baseline="middle").encode(
    text="post_count:Q",
    color=alt.condition(
        alt.datum.post_count > 5, alt.value("white"), alt.value("black")
    ),
)

# Combine the layers
chart = (
    (heatmap + text).properties(width=900, height=200)
    # .configure_legend(orient="top-right")
)
chart

alt.LayerChart(...)

---

## Charts

#### Cumulative plays over time

In [23]:
alt.Chart(df).mark_line(color=color).encode(
    x=alt.X(
        "create_time_est:T", axis=alt.Axis(format="%b. %-d", tickCount=4), title=""
    ),
    y=alt.Y(
        "play_count_cumsum",
        axis=alt.Axis(
            format=".2s",
        ),
        title="",
    ),
).properties(width=300, title=f"{user_name} cumulative TikTok views")

alt.Chart(...)

---

## Exports

#### JSON

In [18]:
# df_posts.to_json(
#     f"data/processed/{user}_all_posts_{today}.json",
#     indent=4,
#     orient="records",
# )

#### CSV

In [19]:
# df_posts.to_csv(f"data/processed/{user_to_export}_all_posts_{today}.csv", index=False)